# Import Libraries

In [2]:
import pandas as pd

# Import Dataset

In [3]:
raw_SPY_intraday_df = pd.read_csv("../Data/SPY_intraday_polygon_2024_2025.csv")

# Data Cleaning

In [4]:
raw_SPY_intraday_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 365406 entries, 0 to 365405
Data columns (total 10 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   timestamp_utc  365406 non-null  object 
 1   open           365406 non-null  float64
 2   high           365406 non-null  float64
 3   low            365406 non-null  float64
 4   close          365406 non-null  float64
 5   volume         365406 non-null  float64
 6   vwap           365406 non-null  float64
 7   ret            365405 non-null  float64
 8   vol_5m         365401 non-null  float64
 9   vol_30m        365376 non-null  float64
dtypes: float64(9), object(1)
memory usage: 27.9+ MB


## Check data types

## Check for missing values

# Exploratory Data Analysis

## Price History and Trends
Visualise market movements and detect structural shifts

## Descriptive Statistics

mean, std, skew, kurtosis, rolling vol, etc.

## Inferential checks
correlations, t-tests, OLS regressions (e.g. return ~ sentiment)

## Stationarity tests

## Lag significance tests

## Normality checks